#  Army Survillience System


The project aims at providing security survillence near unauthorized regions. To protect unwanted people to enter the region. The system detects any unauthorised person and displays unknown and recognise the authorised persons only.

It stores the logs of authorised persons and enable them to enter the zone.

In [ ]:
import cv2


In [ ]:
cv2.__version__

In [ ]:
import cv2, sys, numpy, os, time
count = 0
size = 4
fn_haar = 'haarcascade_frontalface_default.xml'
fn_dir = 'images'
fn_name = input("Who are You?: ")
path = os.path.join(fn_dir, fn_name)

(im_width, im_height) = (112, 92)
haar_cascade = cv2.CascadeClassifier(fn_haar)
webcam = cv2.VideoCapture(0)

print("-------------.....Smile.....Capturing ......----------------------")


while count < 50:
    (rval, im) = webcam.read()
    im = cv2.flip(im, 1, 0)
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    mini = cv2.resize(gray,(gray.shape[1]//size, gray.shape[0]//size))
    faces = haar_cascade.detectMultiScale(mini)
    faces = sorted(faces, key=lambda x: x[3])
    if faces:
        face_i = faces[0]
        (x, y, w, h) = [v * size for v in face_i]
        face = gray[y:y + h, x:x + w]
        face_resize = cv2.resize(face, (im_width, im_height))
        pin=sorted([int(n[:n.find('.')]) for n in os.listdir(path)
               if n[0]!='.' ]+[0])[-1] + 1
        cv2.imwrite('%s/%s.png' % (path, pin), face_resize)
        cv2.rectangle(im, (x, y), (x + w, y + h), (0, 255, 0), 3)
        cv2.putText(im, fn_name, (x - 10, y - 10), cv2.FONT_HERSHEY_PLAIN,
            1,(0, 255, 0))
        time.sleep(0.38)
        count += 1
    cv2.imshow('OpenCV', im)
    if cv2.waitKey(1) & 0xFF == ord('e'):
        break
webcam.release()
cv2.destroyAllWindows()
print(str(count) + " images taken and saved to " + fn_name +" folder in database ")

Who are You?: Ganesh
-------------.....Smile.....Capturing ......----------------------


In [ ]:
pip install opencv-contrib-python

In [ ]:

import cv2, sys, numpy, os
size=4
haar_file = 'haarcascade_frontalface_default.xml'
datasets = 'images'

print('Training...')
# Create a list of images and a list of corresponding names
(images, lables, names, id) = ([], [], {}, 0)
for (subdirs, dirs, files) in os.walk(datasets):
    for subdir in dirs:
        names[id] = subdir
        subjectpath = os.path.join(datasets, subdir)
        for filename in os.listdir(subjectpath):
            path = subjectpath + '/' + filename
            lable = id
            images.append(cv2.imread(path, 0))
            lables.append(int(lable))
        id += 1
(width, height) = (130, 100)

(images, lables) = [numpy.array(lis) for lis in [images, lables]]

model = cv2.face.LBPHFaceRecognizer()
model.train(images, lables)

face_cascade = cv2.CascadeClassifier(haar_file)
webcam = cv2.VideoCapture(0)
while True:
    (_, im) = webcam.read()
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        cv2.rectangle(im, (x, y), (x + w, y + h), (255, 0, 0), 2)
        face = gray[y:y + h, x:x + w]
        face_resize = cv2.resize(face, (width, height))
        # Try to recognize the face
        prediction = model.predict(face_resize)
        cv2.rectangle(im, (x, y), (x + w, y + h), (0, 255, 0), 3)

        if prediction[1] < 100:
            cv2.putText(im, '%s - %.0f' % (names[prediction[0]], prediction[1]), (x - 10, y - 10),
                        cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0))
        else:
            cv2.putText(im, 'UNKNOWN', (x - 10, y - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0,255))

    cv2.imshow('OpenCV', im)
    if cv2.waitKey(1) & 0xFF ==ord('e'):
        break
        
webcam.release()
cv2.destroyAllWindows()